# CH. 9 - Hotspot Analysis
## Exercises

#### Exercise 9.01: Effect of Bandwidth Value

In [ ]:
get_ipython().run_line_magic('matplotlib', 'inline')

import matplotlib.pyplot as plt
import mpl_toolkits.basemap
import numpy
import pandas
import scipy.stats
import seaborn
import sklearn.model_selection
import sklearn.neighbors

seaborn.set()

In [ ]:
# generate x vector for plotting
# the range of the density being estimated

x_vec = numpy.linspace(-30, 30, 10000)[:, numpy.newaxis]

In [ ]:
# generate random sample
# mixture of three normal distributions

numpy.random.seed(42)

vals = numpy.concatenate((
    numpy.random.normal(loc=1, scale=2.5, size=500), 
    numpy.random.normal(loc=10, scale=4, size=500), 
    numpy.random.normal(loc=-12, scale=5, size=500)
))[:, numpy.newaxis]

true_density = (
    (1 / 3) * scipy.stats.norm(1, 2.5).pdf(x_vec[:, 0]) + 
    (1 / 3) * scipy.stats.norm(10, 4).pdf(x_vec[:, 0]) +
    (1 / 3) * scipy.stats.norm(-12, 5).pdf(x_vec[:, 0])
)

In [ ]:
# for plotting:
# (row of plot, column of plot, bandwidth value)

position_bandwidth_vec = [
    (0, 0, 0.1), (0, 1, 0.4), (0, 2, 0.7), 
    (1, 0, 1.0), (1, 1, 1.3), (1, 2, 1.6), 
    (2, 0, 1.9), (2, 1, 2.5), (2, 2, 5.0)
]

In [ ]:
# effect of bandwidth value plot

fig, ax = plt.subplots(3, 3, sharex=True, sharey=True, figsize=(12, 9))
fig.suptitle('The Effect of the Bandwidth Value', fontsize=16)

for r, c, b in position_bandwidth_vec:
    kde = sklearn.neighbors.KernelDensity(bandwidth=b).fit(vals)
    log_density = kde.score_samples(x_vec)

    ax[r, c].hist(vals, bins=50, density=True, alpha=0.5)
    ax[r, c].plot(x_vec[:, 0], numpy.exp(log_density), '-', linewidth=2)
    ax[r, c].set_title('Bandwidth = {}'.format(b))
plt.show()

In [ ]:
# Exercise 9.01 Unit Test

def unittest_exercise_9_01(v1, v2, v3, v4):
    assert len(v1) == 10000, "X_vec length wrong"
    assert int(round(sum(v2)[0])) == -157, "Sum of sampled values wrong"
    assert len(v3) == 10000, "True density length wrong"
    assert len(v4) == 9, "Plot dimensions incorrect"

unittest_exercise_9_01(v1=x_vec, v2=vals, v3=true_density, v4=position_bandwidth_vec)

#### Exercise 9.02: Select Optimal Bandwidth Using Grid Search

In [ ]:
# creating bandwidth vector
# to be used in grid search

bandwidths = 10 ** numpy.linspace(-1, 1, 100)

In [ ]:
# maximize the score
# score = pseudo-log-likelihood

# for time do 10-fold cv instead of leave-one-out

grid = sklearn.model_selection.GridSearchCV(
    estimator=sklearn.neighbors.KernelDensity(),
    param_grid={"bandwidth": bandwidths},
    cv=10 
)
grid.fit(vals)

In [ ]:
# optimal bandwidth value

best_bandwidth = grid.best_params_["bandwidth"]

print(
    "Best Bandwidth Value: {}"
    .format(best_bandwidth)
)

In [ ]:
# histogram of random sample
# 1. true density
# 2. estimated density with optimal bandwidth

fig, ax = plt.subplots(figsize=(14, 10))

ax.hist(vals, bins=50, density=True, alpha=0.5, label='Sampled Values')
ax.fill(x_vec[:, 0], true_density, fc='black', alpha=0.3, label='True Distribution')

log_density = numpy.exp(grid.best_estimator_.score_samples(x_vec))
ax.plot(x_vec[:, 0], log_density, '-', linewidth=2, label='Kernel = Gaussian')

ax.legend(loc='upper right')
plt.show()

In [ ]:
# Exercise 9.02 Unit Test

def unittest_exercise_9_02(env, terms):
    for t in terms:
        filt = [i for i in terms if i == t]
        assert len(filt) == 1, "Object (" + t + ") not in environment"

unittest_exercise_9_02(env=dir(), terms=['grid', 'best_bandwidth'])

#### Exercise 9.03: Effect of Kernel Choice

In [ ]:
# for plotting:
# (row of plot, column of plot, kernel function)

position_kernel_vec = [
    (0, 0, 'gaussian'), (0, 1, 'tophat'), 
    (1, 0, 'epanechnikov'), (1, 1, 'exponential'), 
    (2, 0, 'linear'), (2, 1, 'cosine'), 
]

In [ ]:
# effect of kernel function plot

fig, ax = plt.subplots(3, 2, sharex=True, sharey=True, figsize=(12, 9))
fig.suptitle('The Effect of Different Kernels', fontsize=16)

for r, c, k in position_kernel_vec:
    kde = sklearn.neighbors.KernelDensity(
        kernel=k, 
        bandwidth=best_bandwidth
    ).fit(vals)
    log_density = kde.score_samples(x_vec)

    ax[r, c].hist(vals, bins=50, density=True, alpha=0.5)
    ax[r, c].plot(x_vec[:, 0], numpy.exp(log_density), '-', linewidth=2)
    ax[r, c].set_title('Kernel = {}'.format(k.capitalize()))
plt.show()

In [ ]:
# Exercise 9.03 Unit Test

def unittest_exercise_9_03(v):
    assert len(v) == 6, "Plot not correct"
    assert list(set([isinstance(i, tuple) for i in v]))[0], "Data format incorrect"

unittest_exercise_9_03(v=position_kernel_vec)

#### Exercise 9.04: Each Point as a Distribution

In [ ]:
# function to evaluate the
# gaussian distribution

def eval_gaussian(x, m, b):
    numerator = numpy.exp(
        -numpy.power(x - m, 2) / (2 * numpy.power(b, 2))
    )
    denominator = b * numpy.sqrt(2 * numpy.pi)
    return numerator / denominator

In [ ]:
# plotting one point

m = numpy.array([5.1])
b_vec = [0.1, 0.35, 0.8]

x_vec = numpy.linspace(1, 10, 100)[:, None]

figOne, ax = plt.subplots(2, 3, sharex=True, sharey=True, figsize=(15, 10))

for i, b in enumerate(b_vec):
    ax[0, i].hist(m[:], bins=1, fc='#AAAAFF', density=True)
    ax[0, i].set_title("Histogram: Normed")

    evaluation = eval_gaussian(x_vec, m=m[0], b=b)
    
    ax[1, i].fill(x_vec, evaluation, '-k', fc='#AAAAFF')
    ax[1, i].set_title("Gaussian Dist: b={}".format(b))
plt.show()

In [ ]:
# plotting sixteen points

m = numpy.random.normal(4.7, 0.88, 16)
n = len(m)

b_vec = [0.1, 0.35, 1.1]

x_vec = numpy.linspace(-1, 11, 100)[:, None]

figMulti, ax = plt.subplots(2, 3, sharex=True, sharey=True, figsize=(15, 10))

for i, b in enumerate(b_vec):
    ax[0, i].hist(m[:], bins=n, fc='#AAAAFF', density=True)
    ax[0, i].set_title("Histogram: Normed")
        
    sum_evaluation = numpy.zeros(len(x_vec))
    
    for j in range(n):
        evaluation = eval_gaussian(x_vec, m=m[j], b=b) / n
        sum_evaluation += evaluation[:, 0]
        
        ax[1, i].plot(x_vec, evaluation, '-k', linestyle="dashed")

    ax[1, i].fill(x_vec, sum_evaluation, '-k', fc='#AAAAFF')
    ax[1, i].set_title("Gaussian Dist: b={}".format(b))
plt.show()

In [ ]:
# Exercise 9.04 Unit Test

def unittest_exercise_9_04(env, names):
    plt_list = [i for i in env if i in names]
    assert len(plt_list) == 2, "Number of plots wrong"

unittest_exercise_9_04(env=dir(), names=['figOne', 'figMulti'])

#### Exercise 9.05: Loading Data and Modeling with Seaborn

In [ ]:
# turn housing data into pandas dataframe

df = pandas.read_csv('./california_housing.csv', header=0)

In [ ]:
df.head()

In [ ]:
# median house age in block is less than or equal to 15 years

dfLess15 = df[df['HouseAge'] <= 15.0]
dfLess15 = dfLess15[['Latitude', 'Longitude']]

In [ ]:
dfLess15.head()

In [ ]:
seaborn.jointplot("Longitude", "Latitude", dfLess15, kind="kde")

In [ ]:
# median house age in block is more than 40 years

dfMore40 = df[df['HouseAge'] > 40.0]
dfMore40 = dfMore40[['Latitude', 'Longitude']]

In [ ]:
dfMore40.head()

In [ ]:
seaborn.jointplot("Longitude", "Latitude", dfMore40, kind="kde")

In [ ]:
# perform filter
# plot non-location data

dfLess5 = df[df['HouseAge'] <= 5]

x_vals = dfLess5.Population.values
y_vals = dfLess5.MedInc.values

In [ ]:
fig = plt.figure(figsize=(10, 10))
plt.scatter(x_vals, y_vals, c='black')
plt.xlabel('Population', fontsize=18)
plt.ylabel('Median Income', fontsize=16)

In [ ]:
# run and visualize kernel density estimation
# on non-location data

fig = plt.figure(figsize=(10, 10))
ax = seaborn.kdeplot(
    x_vals, 
    y_vals,
    kernel='gau',
    cmap='Blues', 
    shade=True, 
    shade_lowest=False
)
plt.scatter(x_vals, y_vals, c='black', alpha=0.1)
plt.xlabel('Population', fontsize=18)
plt.ylabel('Median Income', fontsize=18)
plt.title('Density Estimation With Scatterplot Overlay', size=18)

In [ ]:
# Exercise 9.05 Unit Test

def unittest_exercise_9_05(df1, df2, df3):
    assert df1.shape == (20640, 8), "Df1 dimensions incorrect"
    assert df2.shape == (3287, 2), "Df2 dimensions incorrect"
    assert df3.shape == (3878, 2), "Df3 dimensions incorrect"

unittest_exercise_9_05(df1=df, df2=dfLess15, df3=dfMore40)

#### Exercise 9.06: Sklearn and Base Maps

In [ ]:
# set up the data grid for the contour plot

xgrid15 = numpy.sort(list(dfLess15['Longitude']))
ygrid15 = numpy.sort(list(dfLess15['Latitude']))
x15, y15 = numpy.meshgrid(xgrid15, ygrid15)
print("X Grid Component:\n{}\n".format(x15))
print("Y Grid Component:\n{}\n".format(y15))

In [ ]:
# combine x and y grids into pairs
# this is like the x_vec above

xy15 = numpy.vstack([y15.ravel(), x15.ravel()]).T

In [ ]:
# run kernel density estimation

kde15 = sklearn.neighbors.KernelDensity(
    bandwidth=0.05, 
    metric='minkowski',
    kernel='gaussian', 
    algorithm='ball_tree'
)
kde15.fit(dfLess15.values)

In [ ]:
# fit the trained model on the xy grid

log_density = kde15.score_samples(xy15)
density = numpy.exp(log_density)
density = density.reshape(x15.shape)
print("Shape of Density Values:\n{}\n".format(density.shape))

In [ ]:
# visualize the results

fig15 = plt.figure(figsize=(10, 10))
fig15.suptitle(
    """
    Density Estimation:
    Location of Housing Blocks
    Where the Median Home Age <= 15 Years
    """, 
    fontsize=16
)

the_map = mpl_toolkits.basemap.Basemap(
    projection='cyl',
    llcrnrlat=y15.min(), urcrnrlat=y15.max(),
    llcrnrlon=x15.min(),urcrnrlon=x15.max(),
    resolution='c'
)

the_map.drawcoastlines(linewidth=1)
the_map.drawcountries(linewidth=1)
the_map.drawstates(linewidth=1)

levels = numpy.linspace(0, density.max(), 25)
plt.contourf(x15, y15, density, levels=levels, cmap=plt.cm.Reds)

plt.show()

In [ ]:
# set up the data grid for the contour plot

xgrid40 = numpy.sort(list(dfMore40['Longitude']))
ygrid40 = numpy.sort(list(dfMore40['Latitude']))
x40, y40 = numpy.meshgrid(xgrid40, ygrid40)
print("X Grid Component:\n{}\n".format(x40))
print("Y Grid Component:\n{}\n".format(y40))

In [ ]:
# combine x and y grids into pairs
# this is like the x_vec above

xy40 = numpy.vstack([y40.ravel(), x40.ravel()]).T

In [ ]:
# run kernel density estimation

kde40 = sklearn.neighbors.KernelDensity(
    bandwidth=0.05, 
    metric='minkowski',
    kernel='gaussian', 
    algorithm='ball_tree'
)
kde40.fit(dfMore40.values)

In [ ]:
# fit the trained model on the xy grid

log_density = kde40.score_samples(xy40)
density = numpy.exp(log_density)
density = density.reshape(x40.shape)
print("Shape of Density Values:\n{}\n".format(density.shape))

In [ ]:
# visualize the results

fig40 = plt.figure(figsize=(10, 10))
fig40.suptitle(
    """
    Density Estimation:
    Location of Housing Blocks
    Where the Median Home Age > 40 Years
    """, 
    fontsize=16
)

the_map = mpl_toolkits.basemap.Basemap(
    projection='cyl',
    llcrnrlat=y40.min(), urcrnrlat=y40.max(),
    llcrnrlon=x40.min(),urcrnrlon=x40.max(),
    resolution='c'
)

the_map.drawcoastlines(linewidth=1)
the_map.drawcountries(linewidth=1)
the_map.drawstates(linewidth=1)

levels = numpy.linspace(0, density.max(), 25)
plt.contourf(x40, y40, density, levels=levels, cmap=plt.cm.Reds)

plt.show()

In [ ]:
# Exercise 9.06 Unit Test

def unittest_exercise_9_06(env, names):
    plt_list = [i for i in env if i in names]
    assert len(plt_list) == 4, "Number of objects wrong"

unittest_exercise_9_06(env=dir(), names=['kde15', 'fig15', 'kde40', 'fig40'])